https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d

In [1]:
import selenium
from selenium import webdriver
import requests
from bs4 import BeautifulSoup

import time
import io
import os
import hashlib

from PIL import Image

import pandas as pd

In [2]:
#import selenium
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
DRIVER_PATH = './chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [3]:
wd.get('https://google.com')

In [4]:
#search_box = wd.find_element_by_css_selector('input.gLFyf')
#search_box.send_keys('apple')

In [5]:
#wd.quit()

In [6]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results by finding the corresponding css element
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail so that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [7]:
def persist_image(folder_path:str, url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file)#.convert('RGB')
        file_path = os.path.join(folder_path, hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [8]:
def search_and_download(search_term:str, driver_path:str, target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder, elem)

In [9]:
search_term_df = pd.read_csv('./50_food_classes.txt', header=None)
#search_term_df

In [ ]:
for i in search_term_df[0]:
    if i == 'apple':
        search_term = 'single apple fruit'
    else:
        search_term = i
    
    search_and_download(
        search_term = search_term,
        driver_path = DRIVER_PATH,
        number_images = 50
    )

Found: 100 search results. Extracting links from 0:100
Found: 50 image links, done!
SUCCESS - saved https://render.fineartamerica.com/images/rendered/default/canvas-print/8/8/mirror/break/images/artworkimages/medium/1/single-old-wrinkled-bad-yellow-apple-fruit-arletta-cwalina-canvas-print.jpg - as ./images/single_apple_fruit/97b0b3f6b9.jpg
SUCCESS - saved https://previews.123rf.com/images/usersam2007/usersam20071706/usersam2007170600017/80543748-single-ripe-red-apple-fruit-isolated-on-white-background.jpg - as ./images/single_apple_fruit/daa5182692.jpg
ERROR - Could not download https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSNF-8xYpRvQsODtucffg3Of6Ms35PhKVOQTg&usqp=CAU - HTTPSConnectionPool(host='encrypted-tbn0.gstatic.com', port=443): Max retries exceeded with url: /images?q=tbn%3AANd9GcSNF-8xYpRvQsODtucffg3Of6Ms35PhKVOQTg&usqp=CAU (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fab62212ac8>: Failed to establish a new connection: [Errno 60] Op

/Users/iskriyanavasileva/opt/anaconda3/envs/deep-food/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:785: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


SUCCESS - saved https://agfstorage.blob.core.windows.net/misc/FP_com/2020/03/27/GlobalOranges.jpg - as ./images/oranges/6840d1bb9e.jpg
SUCCESS - saved https://i.ndtvimg.com/i/2016-08/orange_625x350_51471855916.jpg - as ./images/oranges/43a9839f65.jpg
ERROR - Could not download https://www.meijer.com/content/dam/meijer/product/0000/00/0003/11/0000000003110_1200.png - HTTPSConnectionPool(host='www.meijer.com', port=443): Read timed out. (read timeout=None)
ERROR - Could not save https://www.meijer.com/content/dam/meijer/product/0000/00/0003/11/0000000003110_1200.png - local variable 'image_content' referenced before assignment
ERROR - Could not download https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRPtq6NVP031G4Fzpl1O7B0hv-uuNexMk0cpw&usqp=CAU - HTTPSConnectionPool(host='encrypted-tbn0.gstatic.com', port=443): Max retries exceeded with url: /images?q=tbn%3AANd9GcRPtq6NVP031G4Fzpl1O7B0hv-uuNexMk0cpw&usqp=CAU (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection obje

KeyboardInterrupt: 

In [ ]:
wd.quit()